In [149]:
import pandas as pd
from icecream import ic
import numpy as np

## Change directory to data (no duplicate rows)

In [150]:
export_dir = 'C:/Users/lavil/source/repos/LukVill/Misc Data/export_no_dup.csv'

export = pd.read_csv(export_dir)

In [151]:
export.head()

,Unnamed: 0,customer_id,account_id,ed_id,event_name,Date,Time
0,0,-784961211,1773350293,12,application_web_approved,2023-03-22,08:45:22
1,1,-784961211,1773350293,19,application_web_view,2023-03-22,13:32:10
2,14,-784961211,1773350293,3,application_web_submit,2023-03-22,13:32:10
3,15,-784961211,1773350293,2,campaign_click,2023-03-22,14:45:22
4,16,-784961211,1773350293,19,application_web_view,2023-07-27,14:57:56


In [152]:
# get max journey length by customer and account
max_journey_len = export.groupby(["customer_id","account_id"]).count().iloc[:,0].max()
max_journey_len

195

In [153]:
# get each unique cust/account
res = export[["customer_id","account_id"]].drop_duplicates().reset_index(drop=True)

# make event column
res["events"] = 0
res["events"] = res["events"].astype(object)

In [154]:
# for each pair, filter data to pair
# then arrange via date and time
# then extract events into list
# once extracted, convert list into np.array
# and set it into res

for i in range(len(res)):
    # get data pair for index
    pair = res.iloc[i,:]
    # extract cust and acct
    cust = pair[0]
    acct = pair[1]
    
    # filter data to pair
    df = export.loc[(export["customer_id"] == cust) & (export["account_id"] == acct)].sort_values(by = ["Date","Time"])
    # extract events into list
    events = df["ed_id"]
    
    # pad events list to reach max journey
    if len(events) < max_journey_len:
        num_zeros = max_journey_len - len(events)
        z_list = np.repeat(0,num_zeros)
        # append list
        events = np.append(events, z_list)
        
    elif len(events) == max_journey_len:
        pass

    else:
        raise Exception("length of event list of customer ", cust, " is less than max journey length ", max_journey_len)
    
    # convert and set
    res.at[i,"events"] = events.astype(object)
    # res is now of unique cust/acct with events


In [157]:
# check output
res.head(n = 15)

,customer_id,account_id,events
0,-784961211,1773350293,"[12, 19, 3, 2, 19, 19, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,15849251,383997507,"[4, 4, 4, 4, 4, 2, 4, 4, 19, 19, 19, 19, 19, 3..."
2,155529381,1786111954,"[2, 12, 3, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
3,-1697747935,-1016455199,"[19, 19, 19, 19, 19, 3, 19, 12, 19, 19, 19, 19..."
4,-986357645,1289564882,"[12, 4, 3, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
5,689736168,900049321,"[12, 19, 3, 1, 4, 24, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,1938053063,1808210098,"[19, 19, 19, 19, 19, 19, 19, 19, 3, 12, 2, 0, ..."
7,-1522106248,-693958153,"[19, 19, 19, 19, 3, 19, 12, 4, 4, 4, 4, 4, 4, ..."
8,1483781929,-47514109,"[12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
9,-484079419,1366678149,"[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 3, 12..."


## SAVE OUTPUT TO THIS FILE PATH

In [159]:
# SAVE OUTPUT

# FOLDER
save_dir = "C:/Users/lavil/source/repos/LukVill/Misc Data"

res.to_csv(save_dir + "/export_prepped.csv")